In [1]:

import tensorflow as tf
from tensorflow.python.ops.rnn_cell import LSTMCell

import numpy as np


class LSTMAutoencoder(object):

    """Basic version of LSTM-autoencoder.
  (cf. http://arxiv.org/abs/1502.04681)
  Usage:
    ae = LSTMAutoencoder(hidden_num, inputs)
    sess.run(ae.train)
  """

    def __init__(
        self,
        hidden_num,
        inputs,
        cell=None,
        optimizer=None,
        reverse=True,
        decode_without_input=False,
        ):
        """
    Args:
      hidden_num : number of hidden elements of each LSTM unit.
      inputs : a list of input tensors with size 
              (batch_num x elem_num)
      cell : an rnn cell object (the default option 
            is `tf.python.ops.rnn_cell.LSTMCell`)
      optimizer : optimizer for rnn (the default option is
              `tf.train.AdamOptimizer`)
      reverse : Option to decode in reverse order.
      decode_without_input : Option to decode without input.
    """

        self.batch_num = inputs[0].get_shape().as_list()[0]
        self.elem_num = inputs[0].get_shape().as_list()[1]

        if cell is None:
            self._enc_cell = LSTMCell(hidden_num)
            self._dec_cell = LSTMCell(hidden_num)
        else:
            self._enc_cell = cell
            self._dec_cell = cell

        with tf.variable_scope('encoder'):
            (self.z_codes, self.enc_state) = tf.contrib.rnn.static_rnn(self._enc_cell, inputs, dtype=tf.float32)

        with tf.variable_scope('decoder') as vs:
            dec_weight_ = tf.Variable(tf.truncated_normal([hidden_num,
                    self.elem_num], dtype=tf.float32), name='dec_weight'
                    )
            dec_bias_ = tf.Variable(tf.constant(0.1,
                                    shape=[self.elem_num],
                                    dtype=tf.float32), name='dec_bias')

            if decode_without_input:
                dec_inputs = [tf.zeros(tf.shape(inputs[0]),
                              dtype=tf.float32) for _ in
                              range(len(inputs))]
                (dec_outputs, dec_state) = tf.contrib.rnn.static_rnn(self._dec_cell, dec_inputs, initial_state=self.enc_state,
                        dtype=tf.float32)
                if reverse:
                    dec_outputs = dec_outputs[::-1]
                dec_output_ = tf.transpose(tf.stack(dec_outputs), [1, 0,
                        2])
                dec_weight_ = tf.tile(tf.expand_dims(dec_weight_, 0),
                        [self.batch_num, 1, 1])
                self.output_ = tf.matmul(dec_output_, dec_weight_) + dec_bias_
            else:

                dec_state = self.enc_state
                dec_input_ = tf.zeros(tf.shape(inputs[0]),
                        dtype=tf.float32)
                dec_outputs = []
                for step in range(len(inputs)):
                    if step > 0:
                        vs.reuse_variables()
                    (dec_input_, dec_state) = \
                        self._dec_cell(dec_input_, dec_state)
                    dec_input_ = tf.matmul(dec_input_, dec_weight_) \
                        + dec_bias_
                    dec_outputs.append(dec_input_)
                if reverse:
                    dec_outputs = dec_outputs[::-1]
                self.output_ = tf.transpose(tf.stack(dec_outputs), [1,
                        0, 2])

        self.input_ = tf.transpose(tf.stack(inputs), [1, 0, 2])
        self.loss = tf.reduce_mean(tf.square(self.input_
                                   - self.output_))

        if optimizer is None:
            self.train = tf.train.AdamOptimizer().minimize(self.loss)
        else:
            self.train = optimizer.minimize(self.loss)

/Users/claudio.gauna/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Basic libraries
import numpy as np
import tensorflow as tf

tf.reset_default_graph()
tf.set_random_seed(2016)
np.random.seed(2016)

# LSTM-autoencoder
#from LSTMAutoencoder import *

# Constants
batch_num = 128
hidden_num = 12
step_num = 8
elem_num = 1
iteration = 10000

# placeholder list
p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num))
p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

cell = tf.nn.rnn_cell.LSTMCell(hidden_num, use_peepholes=True)
ae = LSTMAutoencoder(hidden_num, p_inputs, cell=cell, decode_without_input=True)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iteration):
        """Random sequences.
          Every sequence has size batch_num * step_num * elem_num 
          Each step number increases 1 by 1.
          An initial number of each sequence is in the range from 0 to 19.
          (ex. [8. 9. 10. 11. 12. 13. 14. 15])
        """
        r = np.random.randint(20, size=batch_num).reshape([batch_num, 1, 1])
        r = np.tile(r, (1, step_num, elem_num))
        d = np.linspace(0, step_num, step_num, endpoint=False).reshape([1, step_num, elem_num])
        d = np.tile(d, (batch_num, 1, 1))
        random_sequences = r + d

        (loss_val, _) = sess.run([ae.loss, ae.train], {p_input: random_sequences})
        print('iter %d:' % (i + 1), loss_val)

    (input_, output_) = sess.run([ae.input_, ae.output_], {p_input: r + d})
    print('train result :')
    print('input :', input_[0, :, :].flatten())
    print('output :', output_[0, :, :].flatten())

iter 1: 234.70761
iter 2: 244.35434
iter 3: 241.93172
iter 4: 229.74608
iter 5: 212.09308
iter 6: 220.1326
iter 7: 229.48561
iter 8: 225.89987
iter 9: 209.88054
iter 10: 230.21274
iter 11: 236.53073
iter 12: 206.73788
iter 13: 205.96512
iter 14: 234.29027
iter 15: 210.68942
iter 16: 205.47209
iter 17: 241.96683
iter 18: 234.84303
iter 19: 212.70532
iter 20: 204.16309
iter 21: 210.62968
iter 22: 194.67226
iter 23: 206.13396
iter 24: 223.90436
iter 25: 219.55052
iter 26: 197.01936
iter 27: 212.28357
iter 28: 186.60712
iter 29: 193.78183
iter 30: 194.8486
iter 31: 213.54001
iter 32: 185.17303
iter 33: 207.61563
iter 34: 212.44548
iter 35: 206.16855
iter 36: 173.19386
iter 37: 198.89081
iter 38: 191.00005
iter 39: 180.79373
iter 40: 192.79207
iter 41: 208.30951
iter 42: 180.73752
iter 43: 170.95093
iter 44: 212.67337
iter 45: 187.16866
iter 46: 202.3486
iter 47: 167.93674
iter 48: 189.8483
iter 49: 195.79504
iter 50: 160.42937
iter 51: 190.50381
iter 52: 161.79086
iter 53: 178.30675
iter 5

iter 428: 11.331726
iter 429: 14.869272
iter 430: 12.420179
iter 431: 12.352117
iter 432: 12.419014
iter 433: 14.403565
iter 434: 16.102434
iter 435: 14.834759
iter 436: 13.463644
iter 437: 12.614343
iter 438: 9.708401
iter 439: 11.079498
iter 440: 14.130869
iter 441: 13.086304
iter 442: 15.244721
iter 443: 16.059853
iter 444: 15.790568
iter 445: 13.6183
iter 446: 16.063171
iter 447: 12.319702
iter 448: 13.007202
iter 449: 13.583059
iter 450: 13.782379
iter 451: 12.97835
iter 452: 16.21006
iter 453: 11.56683
iter 454: 13.110527
iter 455: 14.236381
iter 456: 11.442833
iter 457: 12.431398
iter 458: 12.797159
iter 459: 10.262325
iter 460: 12.881631
iter 461: 12.375628
iter 462: 12.266858
iter 463: 16.767982
iter 464: 12.624224
iter 465: 15.4513035
iter 466: 14.992467
iter 467: 15.4102955
iter 468: 10.804251
iter 469: 10.211353
iter 470: 14.376326
iter 471: 11.817365
iter 472: 11.3094
iter 473: 17.06358
iter 474: 15.714102
iter 475: 15.241007
iter 476: 13.7725
iter 477: 9.832312
iter 478: 

iter 860: 4.0419703
iter 861: 3.9498672
iter 862: 4.305156
iter 863: 4.8878636
iter 864: 4.9586535
iter 865: 4.1668973
iter 866: 4.582736
iter 867: 4.075868
iter 868: 4.1902595
iter 869: 4.6344147
iter 870: 5.613659
iter 871: 4.202503
iter 872: 5.064712
iter 873: 4.045126
iter 874: 4.431331
iter 875: 3.5965557
iter 876: 2.4622002
iter 877: 4.828998
iter 878: 3.7840316
iter 879: 4.4779425
iter 880: 4.0608873
iter 881: 4.455306
iter 882: 3.1663787
iter 883: 4.041786
iter 884: 4.5034723
iter 885: 4.022408
iter 886: 3.2776022
iter 887: 4.0043387
iter 888: 5.6121883
iter 889: 3.8314903
iter 890: 3.8160818
iter 891: 4.033228
iter 892: 4.2875957
iter 893: 4.594577
iter 894: 6.0813565
iter 895: 4.268388
iter 896: 3.6066785
iter 897: 5.0634336
iter 898: 4.622139
iter 899: 3.8843462
iter 900: 4.3738413
iter 901: 4.131663
iter 902: 4.280398
iter 903: 3.53904
iter 904: 2.9446702
iter 905: 5.2126503
iter 906: 4.4241934
iter 907: 4.6863265
iter 908: 5.2036166
iter 909: 5.0471077
iter 910: 2.6492863


iter 1270: 1.4596585
iter 1271: 1.4404022
iter 1272: 1.3765061
iter 1273: 1.5835943
iter 1274: 1.5757353
iter 1275: 1.6869439
iter 1276: 2.1342437
iter 1277: 2.139872
iter 1278: 1.1410583
iter 1279: 2.08716
iter 1280: 1.8513522
iter 1281: 1.8382691
iter 1282: 2.1703165
iter 1283: 1.1229506
iter 1284: 1.8252363
iter 1285: 2.055329
iter 1286: 2.7533455
iter 1287: 1.733171
iter 1288: 2.9438236
iter 1289: 2.0860717
iter 1290: 2.2580605
iter 1291: 1.9199764
iter 1292: 1.7429054
iter 1293: 1.7212768
iter 1294: 1.8118341
iter 1295: 1.9600596
iter 1296: 2.0528998
iter 1297: 1.5675553
iter 1298: 1.6697696
iter 1299: 2.0018475
iter 1300: 1.5555255
iter 1301: 1.3080827
iter 1302: 2.0204058
iter 1303: 1.708667
iter 1304: 1.5055426
iter 1305: 1.9824594
iter 1306: 2.0048184
iter 1307: 1.473449
iter 1308: 1.4750502
iter 1309: 1.7575328
iter 1310: 2.0407882
iter 1311: 1.6862818
iter 1312: 1.7761533
iter 1313: 1.7969624
iter 1314: 1.3187919
iter 1315: 2.1726198
iter 1316: 1.8106315
iter 1317: 1.4831715

iter 1662: 0.962307
iter 1663: 0.9824602
iter 1664: 1.1641505
iter 1665: 1.208098
iter 1666: 1.1226233
iter 1667: 0.56153905
iter 1668: 0.6674064
iter 1669: 1.1399392
iter 1670: 1.1392454
iter 1671: 0.79386866
iter 1672: 0.6922332
iter 1673: 0.7268028
iter 1674: 0.886045
iter 1675: 0.8025037
iter 1676: 0.6332576
iter 1677: 0.8655027
iter 1678: 1.0513246
iter 1679: 1.1554147
iter 1680: 1.2130318
iter 1681: 1.051248
iter 1682: 0.92699444
iter 1683: 1.0208886
iter 1684: 1.1301696
iter 1685: 0.8758007
iter 1686: 0.776466
iter 1687: 0.73528063
iter 1688: 0.94639075
iter 1689: 0.9182495
iter 1690: 0.59496975
iter 1691: 1.0082574
iter 1692: 0.97762525
iter 1693: 1.0186114
iter 1694: 0.4965436
iter 1695: 0.96576583
iter 1696: 1.1115072
iter 1697: 0.73196626
iter 1698: 0.7059328
iter 1699: 1.0457828
iter 1700: 0.6949687
iter 1701: 0.8481672
iter 1702: 0.8063894
iter 1703: 0.6427429
iter 1704: 0.8999497
iter 1705: 0.67073894
iter 1706: 1.0079875
iter 1707: 0.9737017
iter 1708: 0.96335185
iter 17

...

iter 8609: 0.001261648
iter 8610: 0.001870855
iter 8611: 0.0015343793
iter 8612: 0.001775465
iter 8613: 0.0012809895
iter 8614: 0.0015287127
iter 8615: 0.0015570421
iter 8616: 0.0018979032
iter 8617: 0.001724324
iter 8618: 0.0014642575
iter 8619: 0.0011220741
iter 8620: 0.0011729214
iter 8621: 0.0015549024
iter 8622: 0.0014717055
iter 8623: 0.0015014479
iter 8624: 0.0014424706
iter 8625: 0.0019247123
iter 8626: 0.0017788304
iter 8627: 0.0009186257
iter 8628: 0.0014042183
iter 8629: 0.001610767
iter 8630: 0.0015386532
iter 8631: 0.0018059395
iter 8632: 0.0011232772
iter 8633: 0.0015129785
iter 8634: 0.0018487268
iter 8635: 0.0018647806
iter 8636: 0.0011932829
iter 8637: 0.0013480795
iter 8638: 0.0015764603
iter 8639: 0.0013727476
iter 8640: 0.0015114322
iter 8641: 0.0011170713
iter 8642: 0.0009391618
iter 8643: 0.0016237138
iter 8644: 0.0016141378
iter 8645: 0.0015298885
iter 8646: 0.0013333982
iter 8647: 0.0011645327
iter 8648: 0.0012361422
iter 8649: 0.0012955636
iter 8650: 0.00135944

iter 8956: 0.0010074152
iter 8957: 0.0014374941
iter 8958: 0.0013689776
iter 8959: 0.00085474644
iter 8960: 0.001105099
iter 8961: 0.0008443725
iter 8962: 0.0012771378
iter 8963: 0.0013136985
iter 8964: 0.001426239
iter 8965: 0.0013117201
iter 8966: 0.0011261549
iter 8967: 0.0011095228
iter 8968: 0.0009776254
iter 8969: 0.0012134828
iter 8970: 0.0013369748
iter 8971: 0.0009381807
iter 8972: 0.0011395686
iter 8973: 0.0013621657
iter 8974: 0.0016798673
iter 8975: 0.001062886
iter 8976: 0.0011071037
iter 8977: 0.001141733
iter 8978: 0.001118995
iter 8979: 0.0009675893
iter 8980: 0.00096353726
iter 8981: 0.0011553998
iter 8982: 0.0009690397
iter 8983: 0.00071661
iter 8984: 0.0010839682
iter 8985: 0.0010629424
iter 8986: 0.0012477817
iter 8987: 0.0013314872
iter 8988: 0.0010545084
iter 8989: 0.0014393909
iter 8990: 0.0017351684
iter 8991: 0.001301392
iter 8992: 0.0012301758
iter 8993: 0.001273271
iter 8994: 0.0010876893
iter 8995: 0.0009729895
iter 8996: 0.0010389837
iter 8997: 0.0013155539

iter 9307: 0.0008720964
iter 9308: 0.00095603603
iter 9309: 0.00074444327
iter 9310: 0.00086049153
iter 9311: 0.0010365739
iter 9312: 0.00075761275
iter 9313: 0.0009218508
iter 9314: 0.0006844076
iter 9315: 0.0009026844
iter 9316: 0.0007373277
iter 9317: 0.0007583564
iter 9318: 0.00088283845
iter 9319: 0.0009610284
iter 9320: 0.0007861034
iter 9321: 0.000919549
iter 9322: 0.0010286254
iter 9323: 0.0008809006
iter 9324: 0.0008018996
iter 9325: 0.00087178615
iter 9326: 0.00073838345
iter 9327: 0.00084472075
iter 9328: 0.0006796763
iter 9329: 0.0009228848
iter 9330: 0.00089771376
iter 9331: 0.0006281794
iter 9332: 0.0008338456
iter 9333: 0.0007783751
iter 9334: 0.00095916603
iter 9335: 0.0008390432
iter 9336: 0.0008103538
iter 9337: 0.0009033186
iter 9338: 0.000991185
iter 9339: 0.00080837286
iter 9340: 0.0009692994
iter 9341: 0.0011012882
iter 9342: 0.0013961775
iter 9343: 0.002017193
iter 9344: 0.0029240497
iter 9345: 0.0023902208
iter 9346: 0.001473821
iter 9347: 0.0010551924
iter 9348

iter 9652: 0.0007863301
iter 9653: 0.0007230062
iter 9654: 0.0008042222
iter 9655: 0.0008800413
iter 9656: 0.00078229944
iter 9657: 0.00054820755
iter 9658: 0.00068850437
iter 9659: 0.0009160129
iter 9660: 0.0007413867
iter 9661: 0.0008436126
iter 9662: 0.00062874047
iter 9663: 0.0007682498
iter 9664: 0.00078150956
iter 9665: 0.0006881795
iter 9666: 0.000574643
iter 9667: 0.0008231534
iter 9668: 0.0007068874
iter 9669: 0.0007418418
iter 9670: 0.00058975245
iter 9671: 0.00077349413
iter 9672: 0.0010778611
iter 9673: 0.0008161954
iter 9674: 0.00095606304
iter 9675: 0.00072304276
iter 9676: 0.00093068875
iter 9677: 0.0009619581
iter 9678: 0.0008047343
iter 9679: 0.00085583434
iter 9680: 0.0007161588
iter 9681: 0.001143628
iter 9682: 0.0008165755
iter 9683: 0.0009053529
iter 9684: 0.00064911036
iter 9685: 0.0007798534
iter 9686: 0.0009191928
iter 9687: 0.0009372403
iter 9688: 0.0009209966
iter 9689: 0.0005623376
iter 9690: 0.0007402501
iter 9691: 0.00089457806
iter 9692: 0.0012524384
iter 

iter 9995: 0.0007714331
iter 9996: 0.0007034588
iter 9997: 0.0005185425
iter 9998: 0.0008229936
iter 9999: 0.00086123036
iter 10000: 0.0009614325
train result :
input : [ 7.  8.  9. 10. 11. 12. 13. 14.]
output : [ 7.003746   8.004268   9.007737  10.004969  11.001642  11.996683
 12.9996395 14.025774 ]


In [10]:
bach_num = 3
r = np.random.randint(10, size=batch_num).reshape([batch_num, 1, 1])

In [13]:
step_num = 2
elem_num = 4
r = np.tile(r, (1, step_num, elem_num))
d = np.linspace(0, step_num, step_num, endpoint=False).reshape([1, step_num, elem_num])
d = np.tile(d, (batch_num, 1, 1))

ValueError: cannot reshape array of size 2 into shape (1,2,4)

In [12]:
r

array([[[0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[3, 3, 3, 3],
        [3, 3, 3, 3]],

       [[3, 3, 3, 3],
        [3, 3, 3, 3]],

       ...,

       [[6, 6, 6, 6],
        [6, 6, 6, 6]],

       [[5, 5, 5, 5],
        [5, 5, 5, 5]],

       [[7, 7, 7, 7],
        [7, 7, 7, 7]]])